In [21]:
import pandas as pd
import quandl
import numpy as np
import math
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression   

df = quandl.get('WIKI/GOOGL')  # get historic of a stock

df = df[["Adj. Open","Adj. High","Adj. Low","Adj. Close","Adj. Volume"]]

df["HL_PCT"] = ((df["Adj. High"] - df["Adj. Close"]) / df["Adj. Close"]) * 100     # High / Close Percentage
df["PCT_change"] = ((df["Adj. Close"] - df["Adj. Open"]) / df["Adj. Open"]) * 100  # Close/Open Precentage

df=df[["Adj. Close","HL_PCT","PCT_change","Adj. Volume"]]

forecast_col = "Adj. Close";
df.fillna(-99999, inplace = True)

forecast_out = int(math.ceil(0.01*len(df)))
# print ("forecast_out =" + str(forecast_out))

df["label"] = df[forecast_col].shift(-forecast_out)

df

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.639972
2004-08-20,54.322689,0.710922,7.227007,22834300.0,69.078238
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,67.839414
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,68.912727
2004-08-25,53.164113,1.886792,1.183658,9188600.0,70.668146
2004-08-26,54.122070,0.037068,2.820391,7094800.0,71.219849
2004-08-27,53.239345,2.326896,-1.803885,6211700.0,72.278116
2004-08-30,51.162935,3.411430,-3.106003,5196700.0,74.810934
2004-08-31,51.343492,1.308977,0.048866,4917800.0,74.199045


In [14]:
# VERSION 1   
df1 = df
df1.dropna(inplace=True)
x = np.array(df1.drop(['label'], 1))  # columns without label 
y = np.array(df1['label'])            # array from column label

x = preprocessing.scale(x)

x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size=0.2) # shuffle and use 20% data as test data 

clf = LinearRegression()
clf.fit(x_train, y_train)
accuracy =  clf.score(x_test, y_test)

accuracy

0.97546690888512577

In [28]:
# VERSION 1   
df2 = df

x = np.array(df2.drop(['label'], 1))  # columns without label 
y = np.array(df2['label'])            # array from column label

x = preprocessing.scale(x)

print(x)
x_lately = x[-forecast_out:]
print (x_lately)
x = x[:-forecast_out:]
x_lately = x[-forecast_out:]
print (x_lately)

[[-1.43832368  2.33958436  0.23152573  4.42841791]
 [-1.42157182 -0.31244291  4.82309448  1.79570797]
 [-1.41928222  2.35448975 -0.80150318  1.24344047]
 ..., 
 [ 2.62113075  0.539492    0.58189391 -0.72623941]
 [ 2.67607896 -0.35366567  1.07552932 -0.7936143 ]
 [ 2.72570824 -0.57262632  0.5611165  -0.78539073]]
[[  2.54674964e+00  -4.57278270e-01  -1.77330287e-01  -7.56491075e-01]
 [  2.56027729e+00  -6.25854519e-01  -1.15455287e-01  -7.69587495e-01]
 [  2.47848319e+00   8.74829313e-01  -1.28447250e+00  -7.62937654e-01]
 [  2.49083816e+00  -8.71737581e-01   1.49537732e-01  -7.87943857e-01]
 [  2.50327690e+00  -6.75893777e-01   3.65411939e-01  -7.93763161e-01]
 [  2.50310937e+00   4.04217589e-01  -4.54376858e-01  -7.38316864e-01]
 [  2.68005768e+00   1.61940486e+00   1.88269182e-01  -3.38759129e-01]
 [  2.67779609e+00  -3.67581679e-01   2.71962156e-01  -6.87933401e-01]
 [  2.67741916e+00  -2.59768008e-01   1.79177006e-02  -7.78971772e-01]
 [  2.71745764e+00  -4.94815374e-01   4.1847569

In [ ]:
x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size=0.2) # shuffle and use 20% data as test data 

clf = LinearRegression()
clf.fit(x_train, y_train)
accuracy =  clf.score(x_test, y_test)